In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.feature_selection import f_regression
from scipy.stats import shapiro, normaltest
import numpy as np
import scipy.stats as sts
from sklearn.datasets import load_iris
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import os

### 1 часть

In [7]:
data = pd.read_excel('LAB_4_DATA_2023.xlsx')
alpha = 0.02
gamma = 0.98
N = len(data)
data

,СТАЖ РАБОТЫ (лет),"ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)","ПОЛ (1-жен, 0- муж)",Зарплата (долл. в час)
0,22.384615,12,0,53.410000
1,8.903846,12,0,8.000000
2,13.250000,15,0,24.000000
3,18.250000,13,0,29.500000
4,13.769231,18,0,32.049999
...,...,...,...,...
535,7.307693,12,1,9.500000
536,19.288462,14,1,17.299999
537,21.230770,12,1,27.520000
538,18.076923,13,1,3.880000


1. Найдите оценки параметров линейной регрессионной модели зависимости ЗАРПЛАТЫ от ВРЕМЕНИ, ЗАТРАЧЕННОГО НА ОБРАЗОВАНИЕ

In [8]:
X1 = data['ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)']  # Извлекаем данные о времени, затраченном на образование
Y1 = data['Зарплата (долл. в час)']  # Извлекаем данные о зарплате в долларах в час

X1 = np.array(X1)  # Преобразуем в массив NumPy
x1 = X1.reshape(len(X1), 1)  # Переформатируем массив для использования в модели

model = LinearRegression().fit(x1, Y1)  # Обучаем модель линейной регрессии
determ = model.score(x1, Y1)  # Вычисляем коэффициент детерминации
print('Коэффициент детерминации:', determ)  # Выводим коэффициент детерминации
print('Пересечение:', model.intercept_)  # Выводим коэффициент пересечения (b₀)
print('Коэффициент наклона:', model.coef_[0], '\n')  # Выводим коэффициент наклона (b₁)

Y1_predicted = model.predict(x1)  # Прогнозируем значения Y на основе X
errors = Y1 - Y1_predicted  # Вычисляем ошибки регрессии

# Выполняем тест Жарка-Бера
print("Тест Джара-Бера")
res_jarq = sts.jarque_bera(errors)
print(res_jarq)

# Проверка на нормальное распределение
if (res_jarq.pvalue > alpha):
  print("Ошибки регрессии имеют нормальное распределение\n")
else:
  print("Ошибки регрессии имеют НЕ нормальное распределение\n")

print("Тест Д’ Агостино")
res_dago = sts.normaltest(errors)
print(res_dago)
if (res_dago.pvalue > alpha):
  print("Ошибки регрессии имеют нормальное распределение")
else:
  print("Ошибки регрессии имеют НЕ нормальное распределение")

Коэффициент детерминации: 0.17249816081531233
Пересечение: -13.933465548912451
Коэффициент наклона: 2.4553205196847245 

Тест Джара-Бера
SignificanceResult(statistic=6301.364567320523, pvalue=0.0)
Ошибки регрессии имеют НЕ нормальное распределение

Тест Д’ Агостино
NormaltestResult(statistic=393.2600421543511, pvalue=4.0240824426047195e-86)
Ошибки регрессии имеют НЕ нормальное распределение


### 1.1. Обладает ли модель высокой объясняющей способностью, или же модель низкого качества (F-тест)

In [97]:
df1 = 1
df2 = N - 2
F_emp = (determ * df2) / ((1 - determ) * df1)  # Эта функция вычисляет значение F-статистики (F_emp), которая используется для оценки значимости регрессионной модели.
F_sts = sts.f.ppf(1 - alpha, df1, df2)
print("Посчитанное значение F эмпирическое", F_emp, "\nЗначение F кр", F_sts)
if F_emp > F_sts:
    print("Модель с высокой объясняющей способностью")
else:
    print("Модель низкого качества")

Посчитанное значение F эмпирическое 112.1496123924933 
Значение F кр 5.444284424851508
Модель с высокой объясняющей способностью


### 2. Найдите оценки параметров линейной регрессионной модели зависимости ЗАРПЛАТЫ от СТАЖА РАБОТЫ, ВРЕМЕНИ, ЗАТРАЧЕННОГО НА ОБРАЗОВАНИЕ, и ПОЛА РАБОТНИКА работника.

In [78]:
X2 = data[['СТАЖ РАБОТЫ (лет)', 'ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)', 'ПОЛ (1-жен, 0- муж)']]  # Выбираем независимые переменные
Y2 = data['Зарплата (долл. в час)']  # Выбираем зависимую переменную
x2 = sm.add_constant(X2)  # Добавляем столбец с постоянным значением к независимым переменным

model2 = sm.OLS(Y2, x2)  # Создаем модель МНК (метод наименьших квадратов)
results = model2.fit()  # Обучаем модель и получаем результаты

print(results.summary())  # Выводим сводку результатов модели

                              OLS Regression Results                              
Dep. Variable:     Зарплата (долл. в час)   R-squared:                       0.241
Model:                                OLS   Adj. R-squared:                  0.237
Method:                     Least Squares   F-statistic:                     56.68
Date:                    Mon, 18 Dec 2023   Prob (F-statistic):           7.74e-32
Time:                            11:15:19   Log-Likelihood:                -2132.2
No. Observations:                     540   AIC:                             4272.
Df Residuals:                         536   BIC:                             4290.
Df Model:                               3                                         
Covariance Type:                nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------

In [79]:
print('Коэффициент детерминации:', results.rsquared)
print('Пересечение:', results.params[0]) # b0
print('Коэффициент наклона:\n', results.params[1:]) # b1, b2, b3

Коэффициент детерминации: 0.240848367627299
Пересечение: -19.691947095344982
Коэффициент наклона:
 СТАЖ РАБОТЫ (лет)                          0.405677
ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)    2.591137
ПОЛ (1-жен, 0- муж)                       -5.909050
dtype: float64


In [80]:
b = results.params  # Получаем параметры модели
X0_ = data['СТАЖ РАБОТЫ (лет)']  # Извлекаем данные о стаже работы
X1_ = data['ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)']  # Извлекаем данные о времени, затраченном на образование
X2_ = data['ПОЛ (1-жен, 0- муж)']  # Извлекаем данные о поле (1-жен, 0- муж)

x0 = np.array(X0_)  # Преобразуем данные стажа работы в массив NumPy
x1 = np.array(X1_)  # Преобразуем данные о времени, затраченном на образование в массив NumPy
x2 = np.array(X2_)  # Преобразуем данные о поле в массив NumPy

XXX = [x0, x1, x2]  # Собираем данные о независимых переменных в массив
XXX = np.array(XXX)  # Преобразуем в массив NumPy
x = XXX.transpose()  # Транспонируем массив для дальнейшего использования
X = sm.add_constant(x)  # Добавляем столбец с постоянным значением к независимым переменным

y_hat = b[0] + b[1] * x0 + b[2] * x1 + b[3] * x2  # Вычисляем предсказанные значения y

print(np.isclose(y_hat, results.predict(), atol=1e-4))  # Проверяем, близки ли вычисленные значения y_hat к предсказанным значениям моделью, с учетом заданной погрешности


[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [81]:
Y2_predicted = results.predict()  # Получаем предсказанные значения зависимой переменной
errors2 = Y2 - Y2_predicted  # Вычисляем ошибки регрессии как разность фактических и предсказанных значений

# Perform Jarque-Bera test
print("Тест Джарка-Бера")
res_jarq = sts.jarque_bera(errors)  # Выполняем тест Джарка-Бера для оценки нормальности ошибок регрессии
print(res_jarq)  # Выводим результат теста

# Проверка на нормальное распределение
if (res_jarq.pvalue > alpha):  # Проверяем результат теста
  print("Ошибки регрессии имеют нормальное распределение\n")  # Выводим сообщение, если ошибки имеют нормальное распределение
else:
  print("Ошибки регрессии имеют НЕ нормальное распределение\n")  # Выводим сообщение, если ошибки не имеют нормальное распределение

print("Тест Д’ Агостино")
res_dago = sts.normaltest(errors2)  # Выполняем тест нормальности ошибок регрессии по методу Д’Агостино
print(res_dago)  # Выводим результат теста

if (res_dago.pvalue > alpha):  # Проверяем результат теста
  print("Ошибки регрессии имеют нормальное распределение")  # Выводим сообщение, если ошибки имеют нормальное распределение
else:
  print("Ошибки регрессии имеют НЕ нормальное распределение")  # Выводим сообщение, если ошибки не имеют нормальное распределение

Тест Джарка-Бера
SignificanceResult(statistic=6301.364567320523, pvalue=0.0)
Ошибки регрессии имеют НЕ нормальное распределение

Тест Д’ Агостино
NormaltestResult(statistic=404.75225464943424, pvalue=1.2857739834290298e-88)
Ошибки регрессии имеют НЕ нормальное распределение


### 2.1. Как в среднем изменится ЗАРПЛАТА, если СТАЖ РАБОТЫ увеличится на 2 года?

In [82]:
print("Изменение ЗАРПЛАТЫ при увеличении стажа на 2 года:", results.params['СТАЖ РАБОТЫ (лет)'] * 2, "\n")

Изменение ЗАРПЛАТЫ при увеличении стажа на 2 года: 0.8113543980273494 



### 2.2. Какую в среднем прибавку к ЗАРПЛАТЕ дает дополнительный год обучения?

In [83]:
print("Прибавка к ЗАРПЛАТЕ за каждый дополнительный год обучения:", results.params['ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет)'], "\n")

Прибавка к ЗАРПЛАТЕ за каждый дополнительный год обучения: 2.5911373236744257 



### 2.3. Проверьте, обладает ли модель из п.3. высокой объясняющей способностью, или же модель низкого качества (Для этого примените F-тест)

In [84]:
m = 3  # Устанавливаем количество оцениваемых параметров
df1 = m  # Задаем число степеней свободы для числителя
df2 = N - m - 1  # Задаем число степеней свободы для знаменателя

RSS = sum((Y2_predicted - Y2.mean())**2)  # Вычисляем сумму квадратов отклонений предсказанных значений от среднего
ESS = sum(errors2**2)  # Вычисляем объясненную сумму квадратов (explained sum of squares)

F_emp = RSS * (len(Y2) - m - 1) / (ESS * m)  # Вычисляем значение F-статистики
F_sts = sts.f.ppf(1 - alpha, df1, df2)  # Определяем критическое значение F-статистики

if (F_emp > F_sts):  # Проводим проверку гипотезы о значимости регрессии с использованием F-критерия
  print("Модель обладает высокой объясняющей способностью")  # Выводим сообщение, если модель обладает высокой объясняющей способностью
else:
  print("Модель низкого качества")  # Выводим сообщение, если модель имеет низкое качество
F_emp, F_sts  # Выводим значения F-статистики и критического значения F-статистики

Модель обладает высокой объясняющей способностью


(56.68376801296465, 3.3063075320369477)

In [85]:
f_test2 = results.f_test("СТАЖ РАБОТЫ (лет) = ВРЕМЯ, ЗАТРАЧЕННОЕ НА ОБРАЗОВАНИЕ (лет) = ПОЛ (1-жен, 0- муж) = 0")  # Выполняем F-тест
f_test_sts = sts.f.ppf(1 - alpha, N - m - 1, m)  # Вычисляем критическое значение F-статистики

print(f"F-тест: {f_test2}")  # Выводим значение F-теста
if (f_test2.statistic > f_test_sts):  # Проверяем является ли значение F-теста больше критического значения
  print("Принимаем H1")  # Выводим сообщение, если принимается альтернативная гипотеза H1
  print("Отражает истинную зависимость\n")
else:
  print("Принимаем H0\n")  # Выводим сообщение, если принимается нулевая гипотеза H0


F-тест: <F test: F=56.683768012964435, p=7.736950333450208e-32, df_denom=536, df_num=3>
Принимаем H1
Отражает истинную зависимость



In [86]:
print("H0: b0 = 0")  # Формулируем нулевую гипотезу для коэффициента b0
print("H1: b0 не равно 0, следовательно значимо")  # Формулируем альтернативную гипотезу для коэффициента b0
Z = X.transpose().dot(X)  # Вычисляем произведение транспонированной матрицы X на матрицу X
invZ = np.linalg.matrix_power(Z, -1)  # Вычисляем обратную матрицу к Z
s_e_2 = (1/(N - m - 1))*sum(errors2**2)  # Вычисляем оценку стандартной ошибки остатков

cob0 = np.sqrt(s_e_2) * np.sqrt(invZ[0, 0])  # Вычисляем оценку стандартной ошибки коэффициента для b0

t_emp = b[0] / cob0  # Вычисляем значение t-статистики
t_crit = sts.t.ppf(1 - alpha/2, N - m -1)  # Вычисляем критическое значение t-статистики

if (abs(t_emp) > t_crit):  # Проводим проверку гипотезы H0 относительно коэффициента b0
  print("Принимаем H1")  # Выводим сообщение, если принимается альтернативная гипотеза H1
  print("Отражает истинную зависимость ")
else:
  print("Принимаем H0")  # Выводим сообщение, если принимается нулевая гипотеза H0
t_emp, t_crit  # Выводим значения t-статистики и критического значения t-статистики

H0: b0 = 0
H1: b0 не равно 0, следовательно значимо
Принимаем H1
Отражает истинную зависимость 


(-4.5157143868064775, 2.333325054701996)

In [87]:
print("H0: b1 = 0")  # Формулируем нулевую гипотезу для коэффициента b1
print("H1: b1 не равно 0, следовательно значимо")  # Формулируем альтернативную гипотезу для коэффициента b1

cob1 = np.sqrt(s_e_2) * np.sqrt(invZ[1,1])  # Вычисляем оценку стандартной ошибки коэффициента для b1

t_emp = b[1] / cob1  # Вычисляем значение t-статистики для коэффициента b1
t_crit = sts.t.ppf(1 - alpha/2, N - m - 1)  # Вычисляем критическое значение t-статистики

if (abs(t_emp) > t_crit):  # Проводим проверку гипотезы H0 относительно коэффициента b1
  print("Принимаем H1")  # Выводим сообщение, если принимается альтернативная гипотеза H1
  print("Отражает истинную зависимость ")
else:
  print("Принимаем H0")  # Выводим сообщение, если принимается нулевая гипотеза H0
t_emp, t_crit  # Выводим значения t-статистики и критического значения t-статистики

H0: b1 = 0
H1: b1 не равно 0, следовательно значимо
Принимаем H1
Отражает истинную зависимость 


(3.149182084162151, 2.333325054701996)

In [88]:
print("H0: b2 = 0")  # Формулируем нулевую гипотезу для коэффициента b2
print("H1: b2 не равно 0, следовательно значимо")  # Формулируем альтернативную гипотезу для коэффициента b2

cob2 = np.sqrt(s_e_2) * np.sqrt(invZ[2,2])  # Вычисляем оценку стандартной ошибки коэффициента для b2

t_emp = b[2] / cob2  # Вычисляем значение t-статистики для коэффициента b2
t_crit = sts.t.ppf(1 - alpha/2, N - m - 1)  # Вычисляем критическое значение t-статистики

if (abs(t_emp) > t_crit):  # Проводим проверку гипотезы H0 относительно коэффициента b2
  print("Принимаем H1")  # Выводим сообщение, если принимается альтернативная гипотеза H1
  print("Отражает истинную зависимость ")
else:
  print("Принимаем H0")  # Выводим сообщение, если принимается нулевая гипотеза H0
t_emp, t_crit  # Выводим значения t-статистики и критического значения t-статистики

H0: b2 = 0
H1: b2 не равно 0, следовательно значимо
Принимаем H1
Отражает истинную зависимость 


(11.33730144066215, 2.333325054701996)

In [89]:
print("H0: b3 = 0")  # Формулируем нулевую гипотезу для коэффициента b3
print("H1: b3 не равно 0, следовательно значимо")  # Формулируем альтернативную гипотезу для коэффициента b3

cob3 = np.sqrt(s_e_2) * np.sqrt(invZ[3,3])  # Вычисляем оценку стандартной ошибки коэффициента для b3

t_emp = b[3] / cob3  # Вычисляем значение t-статистики для коэффициента b3
t_crit = sts.t.ppf(1 - alpha/2, N - m - 1)  # Вычисляем критическое значение t-статистики

if (abs(t_emp) > t_crit):  # Проводим проверку гипотезы H0 относительно коэффициента b3
  print("Принимаем H1")  # Выводим сообщение, если принимается альтернативная гипотеза H1
  print("Отражает истинную зависимость ")
else:
  print("Принимаем H0")  # Выводим сообщение, если принимается нулевая гипотеза H0
t_emp, t_crit  # Выводим значения t-статистики и критического значения t-статистики


H0: b3 = 0
H1: b3 не равно 0, следовательно значимо
Принимаем H1
Отражает истинную зависимость 


(-5.304485921474772, 2.333325054701996)

### 2.4. Значимо ли различаются зарплаты мужчины и женщины при прочих равных условиях (т.е. равных стаже и числу лет, затраченных на образование) ?

In [90]:
print("Прирост ЗАРПЛАТЫ для мужчин по сравнению с женщинами:", results.params['ПОЛ (1-жен, 0- муж)'])

Прирост ЗАРПЛАТЫ для мужчин по сравнению с женщинами: -5.909050234030845


2.5. Можно ли считать, что при начислении зарплаты имеет место дискриминация по гендерному признаку?

In [91]:
print("Так как коэф. b3 имеет значимость => в разности зарплат имеет место дискриминация по гендерному признаку")

Так как коэф. b3 имеет значимость => в разности зарплат имеет место дискриминация по гендерному признаку
